## Drugs.com scraper

In [13]:
import requests
from bs4 import BeautifulSoup, ResultSet, Tag
from typing import Any

print("Setup complete")

Setup complete


In [7]:
content = requests.get("https://www.drugs.com/ibuprofen.html").content
soup = BeautifulSoup(content)

In [8]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Ibuprofen Uses, Dosage &amp; Side Effects - Drugs.com
  </title>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="Ibuprofen (Advil, Motrin) treats minor aches and pains caused by the common cold, headaches, toothaches, back or muscle aches. Includes ibuprofen side effects, interactions and indications." name="description"/>
  <meta content="origin-when-cross-origin" name="referrer"/>
  <meta content="max-image-preview:large" name="robots"/>
  <link href="https://www.drugs.com/ibuprofen.html" rel="canonical"/>
  <link href="/opensearch.xml" rel="search" title="Drugs.com" type="application/opensearchdescription+xml"/>
  <meta content="1470988726468809" property="fb:app_id"/>
  <meta content="https://www.facebook.com/Drugscom" property="article:author"/>
  <meta content="https://www.facebook.com/Drugscom" property="article:publisher"/>
  <meta content="Ibuprofen Uses, Do

In [9]:
cnt = requests.get("https://www.drugs.com/search.php?searchterm=ibuprofen").content
search_soup = BeautifulSoup(cnt)

In [10]:
soup_components: ResultSet[Any] = search_soup.find_all("a", {"class": "ddc-search-result-link-wrap"})

In [12]:
type(soup_components[0])

bs4.element.Tag

In [15]:
def get_soup_items_from_search(search_query: str):
    soup_items = []

    for soup in soup_components:
        title = soup.find("h3", {"class": "ddc-media-title"})
        soup_items.append((title.get_text(), soup.attrs.get("href")))
        print(title.get_text())

    return soup_items

Ibuprofen 
Ibuprofen Prescribing Information
Ibuprofen Chewable Tablets Information Consumer Information
Ibuprofen and Acetaminophen Information Consumer Information
Ibuprofen Suspension: Indications, Side Effects, Warnings Consumer Information
Ibuprofen Tablets: Indications, Side Effects, Warnings Consumer Information
Ibuprofen Capsules: Indications, Side Effects, Warnings Consumer Information
Ibuprofen, Pseudoephedrine, and Chlorpheniramine Consumer Information
Ibuprofen, Phenylephrine, and Chlorpheniramine Consumer Information
GoodSense Ibuprofen PM: Indications, Side Effects, Warnings Consumer Information
Ibuprofen Injection (PDA): Indications, Side Effects, Warnings Consumer Information


[('Ibuprofen ', 'https://www.drugs.com/ibuprofen.html'),
 ('Ibuprofen Prescribing Information',
  'https://www.drugs.com/pro/ibuprofen.html'),
 ('Ibuprofen Chewable Tablets Information Consumer Information',
  'https://www.drugs.com/cdi/ibuprofen-chewable-tablets.html'),
 ('Ibuprofen and Acetaminophen Information Consumer Information',
  'https://www.drugs.com/cdi/ibuprofen-and-acetaminophen.html'),
 ('Ibuprofen Suspension: Indications, Side Effects, Warnings Consumer Information',
  'https://www.drugs.com/cdi/ibuprofen-suspension.html'),
 ('Ibuprofen Tablets: Indications, Side Effects, Warnings Consumer Information',
  'https://www.drugs.com/cdi/ibuprofen-tablets.html'),
 ('Ibuprofen Capsules: Indications, Side Effects, Warnings Consumer Information',
  'https://www.drugs.com/cdi/ibuprofen-capsules.html'),
 ('Ibuprofen, Pseudoephedrine, and Chlorpheniramine Consumer Information',
  'https://www.drugs.com/cdi/ibuprofen-pseudoephedrine-and-chlorpheniramine.html'),
 ('Ibuprofen, Phenylep

In [35]:
def contents_till_next_section(section: Tag) -> list[str]:
    limit_reached = False
    sections_content = []
    next_section = section
    while not limit_reached:
        next_section = next_section.find_next_sibling()
        if next_section is None or next_section.name == "h2":
            limit_reached = True
        else:
            sections_content.append(next_section.get_text(separator="\n\n", strip=True))

    section_name = section.attrs.get("id")
    section_name = section_name.capitalize().replace("-", " ") if section_name is not None else None
    if section_name is None:
        section_name = section.contents[0] if len(section.contents) > 0 else ""
    sections_data = "\n\n".join([x for x in sections_content if x not in [""]])

    return {section_name: sections_data}

def extract_info(sp: BeautifulSoup):
    cnts = sp.find_all("h2", {"class": "ddc-anchor-offset"})
    sections = list(map(lambda x: contents_till_next_section(x), cnts))

    total_data = {}
    for section in sections:
        total_data = total_data | section

    return total_data

contnt = requests.get(
    # "https://www.drugs.com/ibuprofen.html"
    "https://www.drugs.com/mounjaro.html"
).content

sp = BeautifulSoup(contnt)

# sp.prettify()

extract_info(sp)

{'Uses': 'Mounjaro is a once-weekly injection that is FDA approved to be used to decrease blood sugar (blood glucose) in adults who have\n\ntype 2 diabetes\n\n. It should be used together with diet and exercise. Mounjaro has also been shown in clinical trials to be useful for\n\nweight loss\n\n, however it is currently\n\nnot\n\nan\n\nFDA approved weight loss drug.\n\nDiabetes is a long term condition when your blood sugars (HbA1c) become too high because the body does not produce or use insulin normally.\xa0 Overtime high blood sugars can cause serious health problems including\n\nheart disease\n\n,\n\nvision loss\n\nand\n\nkidney disease.\n\nMounjaro works to reduce blood sugar levels by increasing your insulin production and also lowering the amount of sugar your liver makes. It also slows the rate food passes through your body which makes you feel full longer.\n\nMounjaro (Tirzepatide) is the first dual GIP/GLP-1 receptor agonist which means it works on the glucose-dependent insuli

In [24]:
from typing import List, Dict
import aiohttp

async def send_async_request(url, session):
    async with session.get(url) as response:
        resp = await response.read()
        soup_a = BeautifulSoup(resp.decode("utf-8"), features="lxml")

        soup_a.find_all

async def scrape_articles(links: List[str], loop):
    results: List[Dict[str, Any]] = []

    async with aiohttp.ClientSession(conn_timeout=5.0, loop=loop) as session:



SyntaxError: incomplete input (4140639633.py, line 15)